### Imports and settings for this notebool

In [1]:
# general notebook formatting for markdown and plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

In [6]:
# imports for code examples
import os

from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.files.gromos_system.ff.forcefield_system import forcefield_system

# PyGromosToolsDay

## Introduction

This notebook is created for the PyGromosToolsDay 17.02.2022    
Author: Marc Thierry Lehner  

This notbook is part of the PyGromosTools and can be found under:
https://github.com/rinikerlab/PyGromosTools/blob/pygromosDay/examples/PyGromosToolsDay/pyGromosToolsDay.ipynb

### Schedul for the day

1. [Introduction](#Introduction)
    1. [Schedule](#Schedule)
    2. [Tutorial](#Tutorial)
2. [PyGromosTools](#PyGromosTools)
    1. [PyGromosTools](#PyGromosTools)
    2. [PyGromosTools](#PyGromosTools)

## PyGromosTools - a Python package for GROMOS users

PyGromosTools is a Python package for GROMOS users. It tries to provide a easy to use unified interface to the GROMOS users in python. Leading to a better readability and reproducability of code, as well as easier pipelining of GROMOS simulations.

PyGromosTools is a natural grown package which tries to adhere to all GROMOS user's needs.

At the current state PyGromosTools provides:
1. GROMOSxx wrappers
2. GROMOS++ wrappers
3. File handling of all GROMOS file types for automated creation/modification/analysis
4. Automation and file management system gromos_system
5. Simulation Submission and Execution
6. etc.

In the following sections we will see how to use PyGromosTools to automate the creation of GROMOS simulations and how these tools are intended to be used. We will show simple uscase and how to adjust existing classes to fit more complex simulations or new blocks and finally show how you can contribute to PyGromosTools.

### General File Structure

Since the second major relase (TODO: date???) PyGromosTools is focused around a class called `gromos_system`. This class is the main class of PyGromosTools and is used to create, manage and analyse GROMOS simulations.  

`gromos_system` stores information of all files of a simulation and provides methods to create, modify and analyse these files.


<p style="text-align:center;">
    <img src="./figures/gromos_system_overview.png" width=900 alt="gromos_system_overview"/>
    <div style="text-align:center;">Based on B. Ries Thesis</div>
</p> 

`gromos_system` can be create in many different ways. The simplest way is from existing files. However, it also provides the option to be created from only a smiles and a forcfield.  

Although it has to be mentioned, that for all GROMOS types forcefields the correct MTB name is required.  

In the following example we create first a `forcefield_system` that holds all informations about the forcfield we want to use, and then use this `forcefield_system` to create a `gromos_system` from a smiles.   

If we tick the option `adapt_imd_automatically` and `auto_convert` in `gromos_system` we will automatically create not only the topology, but also the coordinate file and a adjusted INPUT file.

In [13]:
work_dir = os.path.abspath("./example_sys/")

In [10]:
ff = forcefield_system(name="off")

Found off: /home/mlehner/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


In [11]:
groSys = Gromos_System(work_folder=work_dir,
                system_name="test_system",
                in_smiles="C1CCCCC1", 
                Forcefield=ff, 
                adapt_imd_automatically=True, 
                auto_convert=True)

This `gromos_system` called `groSys` now contains a topology, coordinate and a input file for Cyclohexane. All automatically generated.  

We can access all these files as python classes by accessing the `groSys.top`, `groSys.cnf` and `groSys.imd` attributes.  
All these classe are instances of the `gromos_file` class and have attributes on their own which are the gromos blocks (and of course somties additional content).  

For example we can check if the new topology has a proper TITLE block by accessing the attribute `groSys.top.TITLE`. And we can see, that as expected the title block was automatically generated.

In [12]:
groSys.top.TITLE

TITLE

	name: 	 hill_formula: C6H12
	----------------------------------------
	| created from OpenForceField topology |
	| use Amber Block for OpenFF topology! |
	----------------------------------------

	 >>> Generated with PyGromosTools (riniker group) <<< 
END

The same is also true for the other files. It's espicially worth to mention that coordinate file is automatically generated, using the RDKit conformer generator.

In [15]:
groSys.cnf.POSITION

POSITION
# 	 
    1 C6H12 C          1    0.141846088   -0.020221766   -0.036618046
    1 C6H12 C          2    0.055102978   -0.136624154    0.012947422
    1 C6H12 C          3   -0.092734970   -0.113323115   -0.020692810
    1 C6H12 C          4   -0.141846060    0.020221627    0.036618086
    1 C6H12 C          5   -0.055102959    0.136624114   -0.012947239
    1 C6H12 C          6    0.092735022    0.113323149    0.020692602
    1 C6H12 H          7    0.247154682   -0.037044655   -0.005387454
    1 C6H12 H          8    0.139249332   -0.016387287   -0.147753466
    1 C6H12 H          9    0.067048734   -0.147903939    0.122959239
    1 C6H12 H         10    0.089243777   -0.230855783   -0.035042794
    1 C6H12 H         11   -0.106467011   -0.113907556   -0.131072375
    1 C6H12 H         12   -0.153521266   -0.196066616    0.021813546
    1 C6H12 H         13   -0.139249269    0.016386798    0.147753518
    1 C6H12 H         14   -0.247154708    0.037044739    0.005387597
    1 

All these blocks the contain classes for the specific fields. In the case of the POSITION block we have a list of atoms. And each atom stores the name, the type and the position. This makes it easy to make adjustments, search for specific atoms or do some general scripted modifiactions to the positions.

<p style="text-align:center;">
    <img src="./figures/gromos_file.png" height=300 alt="gromos file structure"/>
    <div style="text-align:center;">Based on B. Ries Thesis</div>
</p> 

All the fields have theire own attributs and sometimes functions to modify them. For example the `POSITION` block has a `atomP` that store all the information.

In [27]:
atom = groSys.cnf.POSITION[1]
print(atom.atomID, atom.atomType, atom.resName, atom.xp)

2 C C6H12 0.05510297801999234


## Release 3

## Hackathlon

## Final Disscussion